# 理论部分
## Keras是什么？
- 基于 python 的高级神经网络 API
- Francois Chollet 与 2014-2015 年编写 Keras
- 以 Tensorflow、CNTK、Theano 为后端运行，keras 必须有后端才可以运行（现在一般多用 tensorflow）
- 极方便与快速实验，帮助用户以最少的时间验证自己的想法

## TensorFlow-Keras是什么？
- TensorFlow 对 Keras API 规范的实现
- 相对于以 TensorFlow 为后端的 Keras，TensorFlow-Keras 与 Tensorflow 结合更加紧密
- 实现在 tf.keras 空间下

## TensorFlow-Keras 和 Keras 的联系：
- 基于同一套 API（Keras 程序可以通过改导入方式轻松转为 tf.keras 程序；反之可能不成立，因为 tf.keras 有其他特性）
- 相同的 JSON 和 HDF5 模型序列化格式和语义

## TensorFlow-Keras 和 Keras 的区别：
- tf.keras 全面支持 Eager Model（Eager模式是TensorFlow的可交互式的命令行模式，类似于python命令行）
- tf.keras 支持基于 tf.data 的模型训练
- tf.keras 支持 TPU 训练
- tf.keras 支持 tf.distribution 中的分布式策略

# 实战部分
## Keras 搭建分类模型
做个图像分类，数据集用：fashion_mnist。

In [ ]:
# 导入库
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sklearn
import pandas as pd
import os
import sys
import time
import tensorflow as tf

from tensorflow import keras
print(tf.__version__)
print(sys.version_info)
for module in mpl, np, pd, sklearn, tf, keras:
    print(module.__name__, module.__version__)

In [ ]:
# 加载数据
fashion_mnist = keras.datasets.fashion_mnist
(x_train_all, y_train_all), (x_test, y_test) = fashion_mnist.load_data()
# 把数据拆分成训练集和验证集
x_valid, x_train = x_train_all[:5000], x_train_all[5000:]
y_valid, y_train = y_train_all[:5000], y_train_all[5000:]

print(x_valid.shape, y_valid.shape)
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

# 数据归一化
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
# 训练集归一化用 fit_transform
x_train_scaled = scaler.fit_transform(
    x_train.astype(np.float32).reshape(-1, 1)).reshape(-1, 28, 28)
# 验证集测试集归一化用 transform
x_valid_scaled = scaler.transform(
    x_valid.astype(np.float32).reshape(-1, 1)).reshape(-1, 28, 28)
x_test_scaled = scaler.transform(
    x_test.astype(np.float32).reshape(-1, 1)).reshape(-1, 28, 28)

In [ ]:
# 显示单张图片
def show_single_image(img_arr):
    plt.imshow(img_arr, cmap="binary")
    plt.show()

# 调用函数显示第 1 张图片
show_single_image(x_train[0])

# 显示多张图片
def show_imgs(n_rows, n_cols, x_data, y_data, class_name):
    assert len(x_data) == len(y_data)
    assert n_rows * n_cols < len(x_data)
    plt.figure(figsize = (n_cols * 1.4, n_rows * 1.6))
    for row in range(n_rows):
        for col in range(n_cols):
            index = n_cols * row + col
            plt.subplot(n_rows, n_cols, index+1)
            plt.imshow(x_data[index], cmap="binary", interpolation="nearest")
            plt.axis('off')
            plt.title(class_name[y_data[index]])
    plt.show()

class_names = ['T-shirt', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']
# 调用函数显示 15 张图片
show_imgs(3, 5, x_train, y_train, class_names)

In [ ]:
# 模型构建
def create_model():
    # 初始化训练模型
    model = keras.models.Sequential()
    # 模型添加层
    model.add(keras.layers.Flatten(input_shape=[28, 28]))
    for _ in range(4):
        # model.add(keras.layers.Dense(32))
        # # 批归一化
        # model.add(keras.layers.BatchNormalization())
        # # 激活函数
        # model.add(keras.layers.Activation('relu'))
        # 自带归一化的激活函数 Selu
        model.add(keras.layers.Dense(32, activation="selu"))
    # Dropout 层
    # model.add(keras.layers.AlphaDropout(rate=0.5))
    model.add(keras.layers.Dense(10, activation="softmax"))
    """
    其实添加模型可以用另一种写法（直接在模型初始化中设置模型层）：
    model = keras.models.Sequential([
        keras.layers.Flatten(input_shape=[28, 28]),
        keras.layers.Dense(300, activation='relu'),
        keras.layers.Dense(100, activation='relu'),
        keras.layers.Dense(10, activation='softmax')
    ])
    """

    model.compile(loss="sparse_categorical_crossentropy", optimizer="sgd", metrics=["accuracy"])
    return model

cls_model = create_model()
# 查看模型概况
cls_model.summary()

In [ ]:
"""
Keras 回调函数
回调函数在 TensorFlow for Python API 官方文档 的 tf.keras 下的 callbacks 里,
回调函数是作用在训练模型中的操作。涉及到一些 callbacks, 不过常用的是:
1. EarlyStopping: 当被监测的数量不再提升, 则停止训练
2. ModelCheckpoint: 将在每个epoch后保存模型到filepath
3. TensorBoard: 该回调函数是一个可视化的展示器, 将日志写入TensorBoard, 可以通过TensorBoard查看训练过程相关的信息
"""
# 定义文件夹
logdir = './callbacks'
if not os.path.exists(logdir):
    os.mkdir(logdir)
# 定义输出的 Model 文件
out_model_file = os.path.join(logdir, "fashion_mnist_model.h5")
# 定义 callbacks
callbacks = [
    keras.callbacks.TensorBoard(logdir),
    keras.callbacks.ModelCheckpoint(out_model_file, save_best_only=True),  # save_best_only：保存最好的模型，不设置的话，默认保存最近的一个模型
    keras.callbacks.EarlyStopping(patience=5, min_delta=1e-3),
]
# 开始训练
history = cls_model.fit(x_train_scaled, y_train, epochs=10, validation_data=(x_valid_scaled, y_valid), callbacks=callbacks)

# 打印训练的准确率与误差的统计图
def plot_learning_curves(history):
    # 把训练指标数据转成 pd.DataFrame 格式
    pd.DataFrame(history.history).plot(figsize=(8, 5))
    # 显示网格
    plt.grid(True)
    # 设置坐标轴范围
    plt.gca().set_ylim(0, 1)
    plt.show()
# 打印学习曲线图
plot_learning_curves(history)
# 在测试集上进行指标的评估
cls_model.evaluate(x_test_scaled, y_test)

## Keras 搭建回归模型
房价预测问题

In [ ]:
# 数据集
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing()
print(housing.DESCR)
print(housing.data.shape)
print(housing.target.shape)

# 打印了解数据
import pprint
pprint.pprint(housing.data[0:5])
pprint.pprint(housing.target[0:5])

In [ ]:
# 数据集划分
from sklearn.model_selection import train_test_split

x_train_all, x_test, y_train_all, y_test = train_test_split(housing.data, housing.target, random_state=7)
x_train, x_valid, y_train, y_valid = train_test_split(x_train_all, y_train_all, random_state=11)
print(x_train.shape, y_train.shape)
print(x_valid.shape, y_valid.shape)
print(x_test.shape, y_test.shape)

# 数据归一化
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_valid_scaled = scaler.fit_transform(x_valid)
x_test_scaled = scaler.fit_transform(x_test)

In [ ]:
# 模型构建
reg_model = keras.models.Sequential([
    keras.layers.Dense(30, activation='relu', input_shape=x_train_scaled.shape[1:]),
    keras.layers.Dense(1)
])
reg_model.compile(loss='mean_squared_error', optimizer='sgd')
reg_model.summary()


In [ ]:

# 模型训练
callbacks = [keras.callbacks.EarlyStopping(patience=5, min_delta=1e-2)]
history = reg_model.fit(x_train_scaled, y_train, validation_data=(x_valid_scaled, y_valid), epochs=100, callbacks=callbacks)
# 打印学习曲线
def plot_learning_curves(history):
    pd.DataFrame(history.history).plot(figsize=(8, 5))
    plt.grid(True)
    plt.gca().set_ylim(0, 1)
    plt.show()
plot_learning_curves(history)
# 测试评估数据集
reg_model.evaluate(x_test_scaled, y_test)

## Keras 通用模型（Model）
在Keras中有两种深度学习的模型：序列模型（Sequential）和通用模型（Model）。差异在于不同的拓扑结构。
- 序列模型 Sequential：序列模型各层之间是依次顺序的线性关系，模型结构通过一个列表来制定。
- 通用模型Model：通用模型可以设计非常复杂、任意拓扑结构的神经网络，例如有向无环网络、共享层网络等。相比于序列模型只能依次线性逐层添加，通用模型能够比较灵活地构造网络结构，设定各层级的关系。

比如 Wide&Deep 模型，它就不是严格的层级结构，而是由两部分组成的。每一部分都是一个层级结构，所以我们不能用 Sequential 去实现模型了。所以我们用通用模型（Model）的方式进行实现。
- 函数式API
- 子类化

In [ ]:
# 函数式API 实现w&d
def create_model_by_functional_single_input():
    # 单输入
    input = keras.layers.Input(shape=x_train.shape[1:]) # 读取数据
    hidden1 = keras.layers.Dense(30, activation='relu')(input) # (input)之前的可以看作是一个函数，input 是这个函数的输入参数
    hidden2 = keras.layers.Dense(20, activation='relu')(hidden1) # 类似于复合函数 f(x)=h(g(x))
    bn_output = keras.layers.BatchNormalization()(hidden2)
    # 输出之后需要合并模型
    concat = keras.layers.concatenate([input, bn_output]) # 拼接 input 和 hidden2
    output = keras.layers.Dense(1)(concat) # 将拼接好的数据赋给输出层
    # 函数式API 写法需要用 keras.models.Model() 固化模型
    model = keras.models.Model(inputs=[input], outputs=[output])
    model.summary()
    return model

def create_model_by_functional_multi_input():
    # 多输入
    input_wide = keras.layers.Input(shape=[5]) # 读取数据
    input_deep = keras.layers.Input(shape=[6]) # 读取数据
    hidden1 = keras.layers.Dense(30, activation='relu')(input_deep) # (input)之前的可以看作是一个函数，input 是这个函数的输入参数
    hidden2 = keras.layers.Dense(20, activation='relu')(hidden1) # 类似于复合函数 f(x)=h(g(x))
    # 输出之后需要合并模型
    concat = keras.layers.concatenate([input_wide, hidden2]) # 拼接 input 和 hidden2
    output = keras.layers.Dense(1)(concat) # 将拼接好的数据赋给输出层
    # 函数式API 写法需要用 keras.models.Model() 固化模型
    model = keras.models.Model(inputs=[input_wide, input_deep], outputs=[output])
    # model.summary()
    return model


In [ ]:
# 测试多输入
x_train_scaled_wide = x_train_scaled[:, :5]
x_train_scaled_deep = x_train_scaled[:, 2:]
x_valid_scaled_wide = x_valid_scaled[:, :5]
x_valid_scaled_deep = x_valid_scaled[:, 2:]
x_test_scaled_wide = x_test_scaled[:, :5]
x_test_scaled_deep = x_test_scaled[:, 2:]

multi_input_model = create_model_by_functional_multi_input()
multi_input_model.compile(loss="mean_squared_error", optimizer="sgd")
callbacks = [keras.callbacks.EarlyStopping(patience=5, min_delta=1e-2)]
history = multi_input_model.fit([x_train_scaled_wide, x_train_scaled_deep],
    y_train,
    validation_data=([x_valid_scaled_wide, x_valid_scaled_deep], y_valid),
    epochs=100,
    callbacks=callbacks)
# 测试评估集
multi_input_model.evaluate([x_test_scaled_wide, x_test_scaled_deep], y_test)

In [ ]:
# 子类API 实现w&d
class WideDeepModel(keras.models.Model):
    def __init__(self):
        super(WideDeepModel, self).__init__()
        # 定义模型的层次
        self.hidden1_layer = keras.layers.Dense(10, activation='relu')
        self.hidden2_layer = keras.layers.Dense(10, activation='relu')
        self.output_layer = keras.layers.Dense(1)

    def call(self, inputs):
        # 完成模型正向计算
        input_wide, input_deep = inputs
        hidden1 = self.hidden1_layer(input_deep)
        hidden2 = self.hidden2_layer(hidden1)
        concat = keras.layers.concatenate([input_wide, hidden2])
        output = self.output_layer(concat)
        return output

    def build(self, input_shapes):
        # 
        input_wide = keras.layers.Input(shape=[input_shapes[0]], name='input_wide')
        input_deep = keras.layers.Input(shape=[input_shapes[1]], name='input_deep')
        inputs = [input_wide, input_deep]
        return keras.models.Model(inputs=inputs, outputs=self.call(inputs))

wd_model = WideDeepModel()
wd_model.build(input_shapes=[5, 6]).summary()
callbacks = [keras.callbacks.EarlyStopping(patience=5, min_delta=1e-2)]
wd_model.compile(loss="mean_squared_error", optimizer="sgd")
history = multi_input_model.fit([x_train_scaled_wide, x_train_scaled_deep],
    y_train,
    validation_data=([x_valid_scaled_wide, x_valid_scaled_deep], y_valid),
    epochs=100,
    callbacks=callbacks)
# 测试评估集
multi_input_model.evaluate([x_test_scaled_wide, x_test_scaled_deep], y_test)